# Ansys HFSS Interface

## _Versión 0.1_
### Author: Jorge Cárdenas
### Pontificia Universidad de Valparaíso
This development is intended to provide an interface to access HFSS, in order to automate simulation and optimization processes.

## Features

- Automate the re-creation of models in aedt files.
- Create intermediate files to run simulations.
- Run simulations and gather specific metrics from the simualtor.


In [1]:
import sys  
sys.path.insert(0, './src/')

import os
import uuid 


from __future__ import print_function
#from Utilities.SaveAnimation import Video
from dbuilder import Manager as MG



## Setup
<p>In this section we prepare all variables to set the origin and destination folders for our resulting files</p>

In [2]:
ansysPath="C:\\Program Files\\AnsysEM\\AnsysEM21.2\\Win64\\ansysedt.exe"
modelName="box_01_freq_reflect"
modelPath=r"C:/Users/jorge/Documents/Projects Jorge C/DRUIDA PROJECT/POC/dbGeneration_v0/Models/testing-multioutput/"
exportPath= "C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Exports"

project_name="meta-atom_box_01_datageneration"
designName="HFSSDesign1"

#set simulation variables
#This is run to create iterative simulations



In [3]:
#Model builder

#ansysPath,modelName, modelPath,exportPath

Builder=MG.Builder(ansysPath=ansysPath,modelName=modelName,projectName=project_name, designName=designName,modelPath=modelPath, exportPath=exportPath)


In [35]:
#ReCreate a aedt model from script 
#We are supposed to have previously worked our model out and keep a python script 
#to be used as source to create the geometry when required.

Builder.create()

In [4]:
#Generate batch of vectors to simulate
import random

def parameters_Generator(constraints, batch_size, vector_size):
    x = [[0 for i in range(vector_size)] for j in range(batch_size)]

    for vector in x:
        for i, (k, v) in enumerate(constraints.items()):
            vector[i]=random.uniform(constraints[k]["max"], constraints[k]["min"])
            
    return x
    

    

## A full batch of simulation for box-shaped cells

In [5]:

batch=0
iteration=0
simfile_path= "C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\"
simfileName= "intermediateFile.py"

#set reports
reports={
    "ReflectanceTE":"(mag(S(FloquetPort1:1,FloquetPort1:1)))^2",
    "ReflectanceTM":"(mag(S(FloquetPort1:2,FloquetPort1:2)))^2"
    }

constraints = {"xSize":{"max":4, "min":0.3,"nominal":1},
            "ySize":{"max":4, "min":0.3,"nominal":1},
            "zSize":{"max":0.08, "min":0.6,"nominal":0.5},
            "zSustrato":{"max":1.5, "min":0.5,"nominal":1}}

parameters = parameters_Generator(constraints, 3, 4)     


for parameter in parameters:
    
    
    #parameters, batch, iteration, filePath, **kwargs
    #Kwargs
    #AnsoftPath=kwargs['ansoft_path']
    #modelPath=kwargs['model_path'] 
    #outputPath=kwargs['output_path']
    #projectName=kwargs['project_name']
    #simulationID=kwargs['simulation_id']
    #variableName=kwargs['variable_name']
    #value=kwargs['value']
    #units=kwargs['units']
    #design=kwargs['design_name']

    #Create the intermediate simulation fiel
    
    simulation_id=str(uuid.uuid1())

    #set variables to modify
    variable_name='parameters'
    value=str(parameter)

    units="mm"
    
    kwargs={
        "reports":reports,
        "simulation_id":simulation_id,
       "variable_name":variable_name,
        "value" : value,
        "units" : units,
       }
    
    Builder.sim_file('', batch, iteration, simfile_path, **kwargs)
    
    Builder.simulate(simfile_path+simfileName)
    
    iteration+=iteration


"C:/Users/jorge/Documents/Projects Jorge C/DRUIDA PROJECT/POC/dbGeneration_v0/Models/testing-multioutput/meta-atom_box_01_datageneration.aedt"
The new directory is created!
"C:/Users/jorge/Documents/Projects Jorge C/DRUIDA PROJECT/POC/dbGeneration_v0/Models/testing-multioutput/meta-atom_box_01_datageneration.aedt"
The new directory is created!
"C:/Users/jorge/Documents/Projects Jorge C/DRUIDA PROJECT/POC/dbGeneration_v0/Models/testing-multioutput/meta-atom_box_01_datageneration.aedt"
The new directory is created!
